In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
time_step = 14
time_step_out = 3
group = False
epochs = 100
batch_size = 1024

In [3]:
def normalize_types(df):
    for column in df.columns[df.dtypes == 'float64']:
        df[column] = df[column].astype(np.float32)

id = 12
X_train = pd.read_csv(f'./data/X_train_{id}.csv', dtype=np.float32).values
X_val = pd.read_csv(f'./data/X_val_{id}.csv', dtype=np.float32).values
X_test = pd.read_csv(f'./data/X_test_{id}.csv', dtype=np.float32).values

# X_train_ae = pickle.load(open(f'./data/x_train_ae_{id}.pkl', 'rb'))
# X_val_ae = pickle.load(open(f'./data/x_val_ae_{id}.pkl', 'rb'))
# X_test_ae = pickle.load(open(f'./data/x_test_ae_{id}.pkl', 'rb'))

y_train = pd.read_csv(f'./data/y_train_{id}.csv', dtype=np.float32).values
y_val = pd.read_csv(f'./data/y_val_{id}.csv', dtype=np.float32).values
y_test = pd.read_csv(f'./data/y_test_{id}.csv', dtype=np.float32).values

ypm_train = pd.read_csv(f'./data/y_train_{id}.csv', dtype=np.float32).values * 1000
ypm_val = pd.read_csv(f'./data/y_val_{id}.csv', dtype=np.float32).values * 1000
ypm_test = pd.read_csv(f'./data/y_test_{id}.csv', dtype=np.float32).values * 1000

ids_train = pd.read_csv(f'./data/ids_train_{id}.csv').values
ids_val = pd.read_csv(f'./data/ids_val_{id}.csv').values
ids_test = pd.read_csv(f'./data/ids_test_{id}.csv').values

buildings_train = pd.read_csv(f'./data/buildings_train_{id}.csv').values
buildings_val = pd.read_csv(f'./data/buildings_val_{id}.csv').values
buildings_test = pd.read_csv(f'./data/buildings_test_{id}.csv').values

print('train', buildings_train.shape, X_train.shape, y_train.shape, ids_train.shape)
print('val', buildings_val.shape, X_val.shape, y_val.shape, ids_val.shape)
print('test', buildings_test.shape, X_test.shape, y_test.shape, ids_test.shape)

train (302, 1) (87990, 1) (87990, 1) (87990, 2)
val (101, 1) (29938, 1) (29938, 1) (29938, 2)
test (101, 1) (28478, 1) (28478, 1) (28478, 2)


In [4]:
import tensorflow as tf

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [5]:
def max_absolute_error(y_true, y_pred):
    return tf.keras.backend.max(tf.keras.backend.abs(y_true - y_pred))

In [6]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(ylog_train)

# ylog_train = scaler.transform(ylog_train)
# ylog_val = scaler.transform(ylog_val)
# ylog_test = scaler.transform(ylog_test)

In [7]:
def create_sequences(X, y, ids, buildings, time_steps=14, time_steps_out=3, group=False):
    X_output = []
    y_output = []

    for building in buildings:
        if not group and '_grp' in building[0]:
            continue
        
        X_values = X[ids[:,1] == building]
        y_values = y[ids[:,1] == building]

        time_steps_in_hours = time_steps
        _time_steps = time_steps
        _time_steps_out = time_steps_out

        for i in range(len(X_values) - _time_steps + 1):
            if len(y_values) <= (i + _time_steps_out + 1):
                continue
            _X = X_values[i : (i + _time_steps)]
            X_output.append(_X)
            y_output.append(np.sum(y_values[i+1:i+_time_steps_out+1]))

    return np.stack(X_output), np.array(y_output).reshape(-1, 1)


def create_dataset():
    X_train_seq, y_train_seq = create_sequences(X_train[:], ypm_train[:], ids_train[:], buildings_train[:], time_step, time_step_out, group=group)
    X_val_seq, y_val_seq = create_sequences(X_val[:], ypm_val[:], ids_val[:], buildings_val[:], time_step, time_step_out, group=group)
    X_test_seq, y_test_seq = create_sequences(X_test[:], ypm_test[:], ids_test[:], buildings_test[:], time_step, time_step_out, group=group)

    print('train', X_train_seq.shape, y_train_seq.shape)
    print('val', X_val_seq.shape, y_val_seq.shape)
    print('test', X_test_seq.shape, y_test_seq.shape)

    dataset = {
        'X_train': X_train_seq,
        'X_val': X_val_seq,
        'X_test': X_test_seq,
        'y_train': y_train_seq,
        'y_val': y_val_seq,
        'y_test': y_test_seq,
    }
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    output_shape = (y_train_seq.shape[1])
    return dataset, input_shape, output_shape

In [21]:
import datetime
import json
import os
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers

def run_approach(data, input_shape, output_shape, name, learning_rate, beta_1, beta_2, epsilon, epochs=20, batch_size=2048):

    #now = datetime.datetime.now()
    #date = now.strftime("%Y%m%d_%H%M%S")
    path = f'./models/{name}'
    # if not os.path.exists(path):
    #     os.makedirs(path)
    #     os.makedirs(f'{path}/checkpoints')

    # checkpoint_filepath = f'{path}/checkpoints/{{epoch:02d}}.h5'

    # model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    #     filepath=checkpoint_filepath,
    #     save_weights_only=False,
    #     monitor='loss',
    #     mode='min',
    #     save_freq='epoch',
    #     save_best_only=False,
    #     verbose=0)

    optimizer = optimizers.Adam(
        learning_rate=learning_rate,
        beta_1=beta_1,
        beta_2=beta_2,
        epsilon=epsilon)

    model = Sequential([
        layers.LSTM(8, input_shape=input_shape),
        layers.Dense(7, activation='relu'),
        layers.Dense(output_shape)
    ], name=name)

    model.compile(loss='mae', optimizer=optimizer, metrics=['mae', 'mse', 'mape'])

    #model.summary()

    history = model.fit(data['X_train'], data['y_train'], epochs=epochs, validation_data=(data['X_val'], data['y_val']), 
        batch_size=batch_size, shuffle=True, verbose=1)
        # callbacks=[model_checkpoint_callback])

    model.save(f'{path}\model.h5')

    evaluate = model.evaluate(data['X_test'], data['y_test'], verbose=1)

    history_to_save = {}
    history_to_save['history'] = history.history
    history_to_save['evaluate'] = evaluate

    with open(f"{path}\history.json", "w") as history_file:
        history_to_save_json = json.dumps(history_to_save, indent=4)
        history_file.write(history_to_save_json)

    params = {
        'time_step': time_step,
        'time_step_out': time_step_out,
        'model_name': name,
        'model': 'lstm',
        'epochs': epochs,
        'batch_size': batch_size,
        'group': group,
        'y_log': True,
        'y_log_scaler': None,
        'metrics': ['mae', 'mse', 'mape', 'max_absolute_error'],
        'loss': 'mse',
        'data_source': '3_prepare_data_3.ipynb',
        'data_id': 10,
        'buildings_locality': 'all',
        'buildings_type': 'all',
        'measure_period': 'all',
        'parametrs': 'consumption_sqm_log',
        'target' : 'consumption_sqm',
        'hyperparameters': {
            'learning_rate': learning_rate,
            'beta_1': beta_1,
            'beta_2': beta_2,
            'epsilon': epsilon
        }
    }
    
    with open(f"{path}\params.json", "w") as params_file:
        params_json = json.dumps(params, indent=4)
        params_file.write(params_json)


In [10]:
dataset, input_shape, output_shape = create_dataset()

train (84505, 14, 1) (84505, 1)
val (28520, 14, 1) (28520, 1)
test (27112, 14, 1) (27112, 1)


In [17]:
output_shape

1

In [14]:
cases = []
num = 0
for learning_rate in [0.01, 0.001, 0.005]:
    for beta_1 in [0.85, 0.9, 0.95]:
        for beta_2 in [0.99, 0.999, 0.9999]:
            for epsilon in [1e-6, 1e-7, 1e-8]:
                cases.append((num, learning_rate, beta_1, beta_2, epsilon))
                num += 1

cases_df = pd.DataFrame(cases, columns=['num', 'learning_rate', 'beta_1', 'beta_2', 'epsilon'])
cases_df

,num,learning_rate,beta_1,beta_2,epsilon
0,0,0.010,0.85,0.9900,1.000000e-06
1,1,0.010,0.85,0.9900,1.000000e-07
2,2,0.010,0.85,0.9900,1.000000e-08
3,3,0.010,0.85,0.9990,1.000000e-06
4,4,0.010,0.85,0.9990,1.000000e-07
...,...,...,...,...,...
76,76,0.005,0.95,0.9990,1.000000e-07
77,77,0.005,0.95,0.9990,1.000000e-08
78,78,0.005,0.95,0.9999,1.000000e-06
79,79,0.005,0.95,0.9999,1.000000e-07


In [24]:
now = datetime.datetime.now()
date = now.strftime("%Y%m%d_%H%M%S")

for num, learning_rate, beta_1, beta_2, epsilon in cases:
    print(num, "learning_rate:", learning_rate, 'beta_1:', beta_1, 'beta_2:', beta_2, 'epsilon:', epsilon)
    model_name = f'lstm_{time_step_out}d_{time_step}d_gorup24h_log_hyperparams/{date}/{num}'
    run_approach(dataset, input_shape, output_shape, model_name, learning_rate, beta_1, beta_2, epsilon, epochs=30, batch_size=2048)

0 learning_rate: 0.01 beta_1: 0.85 beta_2: 0.99 epsilon: 1e-06
Epoch 1/30
42/42 [==============================] - 3s 26ms/step - loss: 747.5286 - mae: 747.5286 - mse: 975793.1875 - mape: 98.1810 - val_loss: 673.4776 - val_mae: 673.4776 - val_mse: 920606.6875 - val_mape: 92.6363
Epoch 2/30
42/42 [==============================] - 1s 13ms/step - loss: 725.5881 - mae: 725.5881 - mse: 943222.6250 - mape: 91.0213 - val_loss: 643.5041 - val_mae: 643.5041 - val_mse: 878087.6250 - val_mape: 86.0977
Epoch 3/30
42/42 [==============================] - 0s 12ms/step - loss: 686.8986 - mae: 686.8986 - mse: 885347.8125 - mape: 83.8701 - val_loss: 602.7697 - val_mae: 602.7697 - val_mse: 816572.1250 - val_mape: 89.6133
Epoch 4/30
42/42 [==============================] - 0s 12ms/step - loss: 639.0427 - mae: 639.0427 - mse: 811379.9375 - mape: 81.8585 - val_loss: 558.3495 - val_mae: 558.3495 - val_mse: 745959.6875 - val_mape: 101.4465
Epoch 5/30
42/42 [==============================] - 1s 12ms/step - l

C:\Users\pdude\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


848/848 [==============================] - 2s 2ms/step - loss: 56.6502 - mae: 56.6502 - mse: 35141.7969 - mape: 10.0120
1 learning_rate: 0.01 beta_1: 0.85 beta_2: 0.99 epsilon: 1e-07
Epoch 1/30
42/42 [==============================] - 3s 27ms/step - loss: 746.3038 - mae: 746.3038 - mse: 973981.6250 - mape: 97.7318 - val_loss: 669.3613 - val_mae: 669.3613 - val_mse: 915060.6250 - val_mape: 90.5993
Epoch 2/30
42/42 [==============================] - 1s 15ms/step - loss: 719.1666 - mae: 719.1666 - mse: 933706.8125 - mape: 89.4184 - val_loss: 635.8777 - val_mae: 635.8777 - val_mse: 866830.8125 - val_mape: 86.0568
Epoch 3/30
42/42 [==============================] - 1s 15ms/step - loss: 677.7465 - mae: 677.7465 - mse: 871436.8750 - mape: 83.0416 - val_loss: 593.9575 - val_mae: 593.9575 - val_mse: 803042.0625 - val_mape: 91.2337
Epoch 4/30
42/42 [==============================] - 1s 16ms/step - loss: 629.2017 - mae: 629.2017 - mse: 795824.8125 - mape: 82.1813 - val_loss: 550.1704 - val_mae: 5